In [1]:
import argparse
from logging import getLogger
from tqdm import tqdm
import torch
import numpy as np
import pandas as pd
import pickle

from recbole.quick_start import load_data_and_model
from recbole.utils import init_logger, get_model, get_trainer, init_seed, set_color
from recbole.data import create_dataset
from recbole.utils.case_study import full_sort_topk, full_sort_scores

import logging
from logging import getLogger

import torch
import pickle

from recbole.config import Config
from recbole.data import create_dataset, data_preparation, save_split_dataloaders, load_split_dataloaders
from recbole.utils import init_logger, get_model, get_trainer, init_seed, set_color

import copy
from recbole.data.interaction import Interaction, cat_interactions

pd.set_option('display.max_rows', None)


In [2]:
model_file = 'saved/SASRec-meta-Dec-13-2021_20-30-22.pth'  # beijing

checkpoint = torch.load(model_file)
config = checkpoint['config']
# config.final_config_dict['load_col']['inter'].append('item_type_recommend')
# config.final_config_dict['unused_col']['inter'] = ['exposure_time', 'exposure_time_list']
# config.final_config_dict['save_dataset'] = True
# config.final_config_dict['save_dataloaders'] = True

init_seed(config['seed'], config['reproducibility'])
init_logger(config)
logger = getLogger()
logger.info(config)

dataset = create_dataset(config)
beijing = dataset.token2id('cityid', '1')
dataset.inter_feat = dataset.inter_feat[dataset.inter_feat['cityid'] == beijing]
logger.info(dataset)


26 Dec 21:12    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = dataset/pretrain
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None

Training Hyper Parameters:
epochs = 300
train_batch_size = 2048
learner = adam
learning_rate = 0.001
neg_sampling = {'uniform': 1}
eval_step = 1
stopping_step = 500
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'LS': 'valid_and_test'}, 'order': 'TO', 'mode': 'full', 'group_by': 'user'}
repeatable = True
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10]
valid_metric = MRR@10
valid_metric_bigger = True
eval_batch_size = 4096
metric_decimal_place = 4

Dataset Hyper Parameters:
field_separator = 	
seq_separator =  
USER_ID_FIELD = user_id
ITEM_ID_FIELD = item_id
RATING_FIELD = rating
TIME_FIELD = exposure_time


In [3]:
train_data, valid_data, test_data = data_preparation(config, dataset)

init_seed(config['seed'], config['reproducibility'])
model = get_model(config['model'])(config, train_data.dataset).to(config['device'])
model.load_state_dict(checkpoint['state_dict'])
model.load_other_parameter(checkpoint.get('other_parameter'))

26 Dec 21:20    INFO  [Training]: train_batch_size = [2048] negative sampling: [{'uniform': 1}]
26 Dec 21:20    INFO  [Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'LS': 'valid_and_test'}, 'order': 'TO', 'mode': 'full', 'group_by': 'user'}]


In [4]:
trainer = get_trainer(config['MODEL_TYPE'], config['model'])(config, model)
test_result = trainer.evaluate(test_data, load_best_model=False, show_progress=config['show_progress'])
print(test_result)

Evaluate   :   0%|                                                            | 0/8 [00:00<?, ?it/s]:   0%|                                   | 0/8 [00:00<?, ?it/s, GPU RAM: 4.04 G/79.35 G]:  12%|███▍                       | 1/8 [00:00<00:01,  3.54it/s, GPU RAM: 4.04 G/79.35 G]:  12%|███▍                       | 1/8 [00:00<00:01,  3.54it/s, GPU RAM: 9.88 G/79.35 G]:  25%|██████▊                    | 2/8 [00:00<00:01,  3.86it/s, GPU RAM: 9.88 G/79.35 G]:  25%|██████▊                    | 2/8 [00:00<00:01,  3.86it/s, GPU RAM: 9.88 G/79.35 G]:  38%|██████████▏                | 3/8 [00:00<00:01,  4.10it/s, GPU RAM: 9.88 G/79.35 G]:  38%|██████████▏                | 3/8 [00:00<00:01,  4.10it/s, GPU RAM: 9.88 G/79.35 G]:  50%|█████████████▌             | 4/8 [00:00<00:00,  4.49it/s, GPU RAM: 9.88 G/79.35 G]:  50%|█████████████▌             | 4/8 [00:00<00:00,  4.49it/s, GPU RAM: 9.88 G/79.35 G]:  62%|████████████████▉          | 5/8 [00:01<00:00,  4.83it/s, GPU RAM: 9.88 G/79.35 G]:  62%|███

OrderedDict([('recall@10', 0.1253), ('mrr@10', 0.0682), ('ndcg@10', 0.0817), ('hit@10', 0.1253), ('precision@10', 0.0125)])


In [3]:
trainer = get_trainer(config['MODEL_TYPE'], config['model'])(config, model)
test_result = trainer.evaluate(test_data, load_best_model=False, show_progress=config['show_progress'])
print(test_result)

Evaluate   :   0%|                                                            | 0/1 [00:00<?, ?it/s]:   0%|                                   | 0/1 [00:00<?, ?it/s, GPU RAM: 1.64 G/79.35 G]: 100%|███████████████████████████| 1/1 [00:00<00:00,  8.77it/s, GPU RAM: 1.64 G/79.35 G]: 100%|███████████████████████████| 1/1 [00:00<00:00,  8.61it/s, GPU RAM: 1.64 G/79.35 G]


OrderedDict([('recall@10', 0.1576), ('mrr@10', 0.0984), ('ndcg@10', 0.1124), ('hit@10', 0.1576), ('precision@10', 0.0158)])


In [4]:
bj_struct = test_result.struct
bj_struct

In [5]:
model_file = 'saved/SASRec-Dec-14-2021_19-53-21.pth'  # shanghai

checkpoint = torch.load(model_file)

init_seed(config['seed'], config['reproducibility'])
model = get_model(config['model'])(config, train_data.dataset).to(config['device'])
model.load_state_dict(checkpoint['state_dict'])
model.load_other_parameter(checkpoint.get('other_parameter'))

trainer = get_trainer(config['MODEL_TYPE'], config['model'])(config, model)
test_result = trainer.evaluate(test_data, load_best_model=False, show_progress=config['show_progress'])
print(test_result)

Evaluate   :   0%|                                                            | 0/1 [00:00<?, ?it/s]:   0%|                                   | 0/1 [00:00<?, ?it/s, GPU RAM: 2.24 G/79.35 G]: 100%|███████████████████████████| 1/1 [00:00<00:00,  1.72it/s, GPU RAM: 2.24 G/79.35 G]: 100%|███████████████████████████| 1/1 [00:00<00:00,  1.59it/s, GPU RAM: 2.24 G/79.35 G]


OrderedDict([('recall@10', 0.1638), ('mrr@10', 0.0997), ('ndcg@10', 0.1149), ('hit@10', 0.1638), ('precision@10', 0.0164)])


In [6]:
sh_struct = test_result.struct
sh_struct

In [9]:
bj_topk = bj_struct['rec.topk']

In [10]:
sh_topk = sh_struct['rec.topk']

In [17]:
bj_topk[0].nonzero().squeeze(-1)

tensor([ 1, 10])

In [18]:
sh_topk[0].nonzero().squeeze(-1)

tensor([10])

In [19]:
merge_topk = bj_topk.clone()
merge_topk

tensor([[0, 1, 0,  ..., 0, 0, 1],
        [0, 0, 0,  ..., 0, 0, 1],
        [1, 0, 0,  ..., 0, 0, 1],
        ...,
        [0, 0, 0,  ..., 0, 0, 1],
        [0, 0, 0,  ..., 0, 0, 1],
        [0, 0, 0,  ..., 0, 0, 1]])

In [22]:
bj_topk

tensor([[0, 1, 0,  ..., 0, 0, 1],
        [0, 0, 0,  ..., 0, 0, 1],
        [1, 0, 0,  ..., 0, 0, 1],
        ...,
        [0, 0, 0,  ..., 0, 0, 1],
        [0, 0, 0,  ..., 0, 0, 1],
        [0, 0, 0,  ..., 0, 0, 1]])

In [23]:
sh_topk

tensor([[0, 0, 0,  ..., 0, 0, 1],
        [0, 0, 0,  ..., 0, 0, 1],
        [1, 0, 0,  ..., 0, 0, 1],
        ...,
        [0, 0, 0,  ..., 0, 0, 1],
        [0, 0, 0,  ..., 0, 0, 1],
        [0, 0, 0,  ..., 0, 0, 1]])

In [21]:
for i in range(len(bj_topk)):
    bj = bj_topk[i].nonzero().squeeze(-1)[0]
    sh = sh_topk[i].nonzero().squeeze(-1)[0]
    if bj > sh:
        merge_topk[i] = sh_topk[i]
merge_topk

tensor([[0, 1, 0,  ..., 0, 0, 1],
        [0, 0, 0,  ..., 0, 0, 1],
        [1, 0, 0,  ..., 0, 0, 1],
        ...,
        [0, 0, 0,  ..., 0, 0, 1],
        [0, 0, 0,  ..., 0, 0, 1],
        [0, 0, 0,  ..., 0, 0, 1]])

In [25]:
merge_struct = copy.deepcopy(bj_struct)
merge_struct.set('rec.topk', merge_topk)

In [26]:
merge_result = trainer.evaluator.evaluate(merge_struct)

In [27]:
merge_result

OrderedDict([('recall@10', 0.1964),
             ('mrr@10', 0.1144),
             ('ndcg@10', 0.1338),
             ('hit@10', 0.1964),
             ('precision@10', 0.0196)])

In [29]:
merge_result.values()

odict_values([0.1964, 0.1144, 0.1338, 0.1964, 0.0196])

In [30]:
test_data.dataset

pretrain
The number of users: 5065
Average actions of users: 1.0
The number of items: 15476
Average actions of items: 1.4195833333333334
The number of inters: 3407
The sparsity of the dataset: 99.99565355725785%
Remain Fields: ['cityid', 'user_id', 'item_id', 'item_type_recommend', 'exposure_time', 'cityid_list', 'item_id_list', 'item_type_recommend_list', 'exposure_time_list', 'item_length', 'neg_item_id']

In [32]:
dataset

pretrain
The number of users: 5065
Average actions of users: 17.380529225908372
The number of items: 15476
Average actions of items: 5.687560581583199
The number of inters: 88015
The sparsity of the dataset: 99.88771583271183%
Remain Fields: ['cityid', 'user_id', 'item_id', 'item_type_recommend', 'exposure_time']